## scrape single pages from the Hungarian Film Archive
http://mandarchiv.hu/tart/jatekfilm

In [4]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
import random



In [8]:
# 

USER_AGENTS = [
    # Chrome
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
    # Firefox
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:25.0) Gecko/20100101 Firefox/25.0',
    # Opera
    'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
    # Safari
    'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25',
    # Internet Explorer
    'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
]

def get_header(agents):
    return {'User-agent': random.choice(agents)}

In [15]:
url = 'http://mandarchiv.hu/tart/jatekfilm?name=jatekfilm&action=film&id=255012030'
response = requests.get(url, headers=get_header(USER_AGENTS))
soup = BeautifulSoup(response.content, "html.parser")